In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

# Load data, set input and output 

In [15]:
# load csv 
position_data_parent = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_006'
position_data_file = r'subset_for_memory_c\BW-0340\2024_07_17\000_raw_pose_data\gait_vertical_FW_1_mediapipe.csv'
full_position_path = os.path.join(position_data_parent, position_data_file)
print(full_position_path)
df = pd.read_csv(full_position_path)

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_006\subset_for_memory_c\BW-0340\2024_07_17\000_raw_pose_data\gait_vertical_FW_1_mediapipe.csv


In [16]:
# set file path to save plots in same folder as raw data 
# use position data file path and data file to reconstruct ifle path 

head, tail = os.path.split(full_position_path)

# create folder to save plots in 

# which data source? 
if '_yolo' in tail: 
    plot_folder = os.path.join(head, 'yolo_plots')
elif 'mediapipe_world' in tail:
    plot_folder = os.path.join(head, 'mp_world_plots') 
else: 
    plot_folder = os.path.join(head, 'mp_pose_plots') 
print(f'plot_folder: {plot_folder}')

# check path exits 
if os.path.exists(plot_folder):
    print(f"'plot_folder' exists")
else:
    print(f"'plot_folder' does not exist - making new directory")
    os.mkdir(plot_folder) 


plot_folder: C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_006\subset_for_memory_c\BW-0340\2024_07_17\000_raw_pose_data\mp_pose_plots
'plot_folder' exists


In [17]:
# which task - FW or PWS 
if 'FW' in tail: 
    plot_folder_full = os.path.join(plot_folder, 'FW')
elif 'PWS' in tail: 
    plot_folder_full = os.path.join(plot_folder, 'PWS')
print(f'plot_folder_full: {plot_folder_full}')

# check path exits 
if os.path.exists(plot_folder_full):
    print(f"'plot_folder_full' exists")
else:
    print(f"'plot_folder_full' does not exist - making new directory")
    os.mkdir(plot_folder_full) 

plot_folder_full: C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_006\subset_for_memory_c\BW-0340\2024_07_17\000_raw_pose_data\mp_pose_plots\FW
'plot_folder_full' exists


In [18]:
## set up mediapipe drawing 

In [19]:
# set up mediapipe drawing 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Re-format pose data 
group by frame 

In [20]:
# print landmark data 
# currently saved as one row per label for each frame. 32 rows per frame 
df.tail()

,Unnamed: 0,X,Y,Z,vis,pres,frame,label
24085,28,0.406290,1.299953,1.967344,0.113780,0.0,729,right_ankle
24086,29,0.602154,1.362461,1.458198,0.251936,0.0,729,left_heel
24087,30,0.456179,1.343314,2.057858,0.195854,0.0,729,right_heel
24088,31,0.362201,1.387603,1.121975,0.119417,0.0,729,left_foot_index
24089,32,0.199721,1.359144,1.824795,0.092281,0.0,729,right_foot_index


In [21]:
# remove inf values - replace with nan for min and max calculation 
df = df.replace([np.inf, -np.inf], np.nan)

# Set min and max for plots 
xmin = df['X'].min(skipna = True)
xmax = df['X'].max(skipna = True)

ymin = df['Y'].min(numeric_only = True)
ymax = df['Y'].max(numeric_only = True)

In [22]:
# Group landmark data by frame
grouped = df.groupby("frame")
print(grouped.get_group(30))

      Unnamed: 0         X         Y         Z       vis  pres  frame  \
990            0  0.540558  0.107232  0.209488  0.995730   0.0     30   
991            1  0.519533  0.085306  0.048300  0.996921   0.0     30   
992            2  0.503457  0.085941  0.048307  0.997167   0.0     30   
993            3  0.486991  0.086419  0.048102  0.996818   0.0     30   
994            4  0.567183  0.084483  0.065017  0.995759   0.0     30   
995            5  0.584722  0.084977  0.064965  0.995489   0.0     30   
996            6  0.603381  0.085064  0.064196  0.994377   0.0     30   
997            7  0.470329  0.102276 -0.472424  0.997903   0.0     30   
998            8  0.638719  0.098830 -0.395291  0.997001   0.0     30   
999            9  0.518840  0.131031  0.056429  0.998077   0.0     30   
1000          10  0.572790  0.129469  0.080060  0.997672   0.0     30   
1001          11  0.285851  0.243211 -0.709117  0.999019   0.0     30   
1002          12  0.809850  0.236191 -0.499592  0.9

In [23]:
# Get landmark data for each frame
for frame_idx in grouped.groups:

    # initiate plot 
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111)

    # get current group of position data for this frame 
    group = grouped.get_group(frame_idx)
    
    # save all landmarks for this frame  
    landmarks = []

    for _, row in group.iterrows(): # each row = one landmark 
        landmark = landmark_pb2.NormalizedLandmark(
            x=row["X"], y=row["Y"], z=row["Z"], visibility=row["vis"]
        )
        landmarks.append(landmark) # append all landmarks for this frame to landmarks  

    # Extract x and y coordinates 
    x_vals = [lm.x for lm in landmarks]
    y_vals = [lm.y for lm in landmarks]

    # plot 2D 
    ax.scatter(x_vals, y_vals, c='red', s=20)

    # draw connections if all landmarks are present 
    if len(landmarks) == 33:
        for connection in mp_pose.POSE_CONNECTIONS:
            start_idx, end_idx = connection
            x = [landmarks[start_idx].x, landmarks[end_idx].x]
            y = [landmarks[start_idx].y, landmarks[end_idx].y]
            ax.plot(x, y, c='black')
       
    # set consistent limits throughout all plots 
    ax = fig.axes[0]  
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymax, ymin) # flip because pose y is inverse 
    ax.set_aspect('equal') # x and y on same scale 
    
    # Set axis labels
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')

    # draw box around 0 to 1 and 0 to 1 
        # MP "guessing" if outside these bounds 
    ax.hlines(y=1, xmin=0, xmax=1, color='grey', linestyle='-.', alpha = 0.5)
    ax.hlines(y=0, xmin=0, xmax=1, color='grey', linestyle='-.', alpha = 0.5)
    ax.vlines(x = 1, ymin=1, ymax=0, color='grey', linestyle='-.', alpha = 0.5) 
    ax.vlines(x = 0, ymin=1, ymax=0, color='grey', linestyle='-.', alpha = 0.5) 

    # add frame number 

    fig.savefig(os.path.join(plot_folder_full, 
                            f"frame_{frame_idx:06d}.png")
               )
   # plt.show()
    plt.close()


## Create video from files 

In [24]:
# all files in folder 
image_file_names = sorted(os.listdir(plot_folder_full))

In [25]:
# Read the first image to get the frame size
first_image_path = os.path.join(plot_folder_full, image_file_names[0])
frame = cv2.imread(first_image_path)
height, width, layers = frame.shape
frame_size = (width, height)

In [26]:
print(head)

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_006\subset_for_memory_c\BW-0340\2024_07_17\000_raw_pose_data


In [27]:
# Define the codec and create VideoWriter object
fps = 30
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for .avi
out = cv2.VideoWriter(os.path.join(plot_folder_full, 
                                   'video_plots.mp4'),
                      fourcc, fps, frame_size)

# Write each image to the video
for image_name in image_file_names:
    image_path = os.path.join(plot_folder_full, image_name)
    frame = cv2.imread(image_path)
    out.write(frame)

# Release the writer
out.release()
print(f"Video saved to {plot_folder_full}")

Video saved to C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_outputs_006\subset_for_memory_c\BW-0340\2024_07_17\000_raw_pose_data\mp_pose_plots\FW
